In [1]:
!python -V

Python 3.11.4


In [1]:
import pandas as pd
import requests
from io import StringIO,BytesIO

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.metrics import mean_squared_error

In [5]:
import mlflow

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("nyc-taxi-experiment")

2025/05/08 13:32:57 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/349002529353381323', creation_time=1746685977161, experiment_id='349002529353381323', last_update_time=1746685977161, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
def read_dataframe(url):
    # Fetch binary Parquet content from URL
    response = requests.get(url)
    response.raise_for_status()

    # Read into DataFrame directly from memory
    data = BytesIO(response.content)
    df = pd.read_parquet(data)

    # Parse datetime columns (column names may vary — adjust as needed)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    # Compute duration in minutes
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # Ensure categorical IDs are strings
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


In [7]:
urls = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet"
]

df_train = read_dataframe(urls[0])
df_val = read_dataframe(urls[1])

In [8]:
len(df_train), len(df_val)

(73908, 61921)

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

np.sqrt(mean_squared_error(y_val, y_pred))

np.float64(7.758714882773319)

In [14]:
with open('mlruns/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Pathorn")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="mlruns/models/lin_reg.bin", artifact_path="models_pickle")

🏃 View run spiffy-penguin-395 at: http://127.0.0.1:5000/#/experiments/349002529353381323/runs/1e71c4d50e014cd5a8efdb6a1d0644cf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/349002529353381323


In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/ppathorn/Documents/GitHub/mlops_zoomcamp_2025/.venv/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [13:39:09] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:6.76148                           
[1]	validation-rmse:6.72150                           
[2]	validation-rmse:6.70080                           
[3]	validation-rmse:6.68976                           
[4]	validation-rmse:6.67356                           
[5]	validation-rmse:6.66415                           
[6]	validation-rmse:6.65502                           
[7]	validation-rmse:6.65228                           
[8]	validation-rmse:6.64726                           
[9]	validation-rmse:6.64450                           
[10]	validation-rmse:6.64090                          
[11]	validation-rmse:6.63553                          
[12]	validation-rmse:6.63267                          
[13]	validation-rmse:6.62980                          
[14]	validation-rmse:6.62844                          
[15]	validation-rmse:6.62640                          
[16]	validation-rmse:6.62244                          
[17]	validation-rmse:6.61891                          
[18]	valid

/Users/ppathorn/Documents/GitHub/mlops_zoomcamp_2025/.venv/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [13:39:29] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.08686                                                    
[1]	validation-rmse:7.68095                                                    
[2]	validation-rmse:7.09092                                                    
[3]	validation-rmse:6.84242                                                    
[4]	validation-rmse:6.72545                                                    
[5]	validation-rmse:6.67076                                                    
[6]	validation-rmse:6.64263                                                    
[7]	validation-rmse:6.62498                                                    
[8]	validation-rmse:6.61117                                                    
[9]	validation-rmse:6.60407                                                    
[10]	validation-rmse:6.59689                                                   
[11]	validation-rmse:6.59519                                                   
[12]	validation-rmse:6.59188            

...

[0]	validation-rmse:11.10228                                                   
[1]	validation-rmse:10.18930                                                   
[2]	validation-rmse:9.44482                                                    
[3]	validation-rmse:8.84266                                                    
[4]	validation-rmse:8.35925                                                    
[5]	validation-rmse:7.97405                                                    
[6]	validation-rmse:7.66788                                                    
[7]	validation-rmse:7.42467                                                    
[8]	validation-rmse:7.23434                                                    
[9]	validation-rmse:7.08515                                                    
[10]	validation-rmse:6.96675                                                   
[11]	validation-rmse:6.87270                                                   
[12]	validation-rmse:6.79819            

/Users/ppathorn/Documents/GitHub/mlops_zoomcamp_2025/.venv/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [13:54:17] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:10.42578                                                   
[1]	validation-rmse:9.16682                                                    
[2]	validation-rmse:8.30141                                                    
[3]	validation-rmse:7.71765                                                    
[4]	validation-rmse:7.32802                                                    
[5]	validation-rmse:7.06958                                                    
[6]	validation-rmse:6.89860                                                    
[7]	validation-rmse:6.78368                                                    
[8]	validation-rmse:6.70700                                                    
[9]	validation-rmse:6.65209                                                    
[10]	validation-rmse:6.61470                                                   
[11]	validation-rmse:6.58561                                                   
[12]	validation-rmse:6.56491            

/Users/ppathorn/Documents/GitHub/mlops_zoomcamp_2025/.venv/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [13:55:06] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.72484                                                   
[1]	validation-rmse:11.27397                                                   
[2]	validation-rmse:10.85829                                                   
[3]	validation-rmse:10.47558                                                   
[4]	validation-rmse:10.12359                                                   
[5]	validation-rmse:9.80101                                                    
[6]	validation-rmse:9.50471                                                    
[7]	validation-rmse:9.23308                                                    
[8]	validation-rmse:8.98542                                                    
[9]	validation-rmse:8.75772                                                    
[10]	validation-rmse:8.55026                                                   
[11]	validation-rmse:8.36127                                                   
[12]	validation-rmse:8.19036            

In [18]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("mlruns/models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("mlruns/models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[11:40:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	valid

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
